In [ ]:
import json
import pandas as pd
pd.set_option('display.max_rows', 1000)

#file contains list of interested genes type json: this can be changed
filename_s = 'file\extract\CPC013_HCC515_6H_BRD-K05104363_10.0.json'

with open(filename_s, "rb") as data_in_s:
    data_s = json.load(data_in_s)

df_s = pd.DataFrame(list(data_s.items()), columns = ['variable', 'value']) 
upgene= df_s['value'][df_s['variable']=='upGenes']
dngene= df_s['value'][df_s['variable']=='dnGenes']
my_list_compare = ['genes']


for item_s in upgene:
    my_set_s = set(item_s)
    my_new_list_up = list(my_set_s)
    for i in my_new_list_up:
        my_list_compare.append(i)

for item in dngene:
    my_set = set(item)
    my_new_list_dn = list(my_set)
    for i in my_new_list_dn:
        my_list_compare.append(i)
    

In [ ]:
from bsonstream import KeyValueBSONInput
import csv
import bson 
import json
from bson import ObjectId
import numpy as np
from string import punctuation

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

with open('file\cpcd-gse70138.bson', 'rb') as f:
    #name of .csv file
    with open('name.csv', 'w') as outfile:
        write = csv.writer(outfile)
        write.writerow(my_list_compare)
        stream = KeyValueBSONInput(fh=f)
        for dict_data in stream:
            del dict_data['_id']
            #name of row in .csv
            name = str(dict_data['batch'])+'_' +str(dict_data['pert_id'])+'_' +str(dict_data['pert_dose'])
            #get rid of file that does not contian a list of genes
            if dict_data.get('upGenes') != None:
                output = np.zeros(len(my_list_compare), dtype=object);
                output[0] = name
                index = 1
                #comparing interested up regulated gene set and file from .bson
                for i in my_new_list_up:
                    for j in dict_data.get('upGenes'):
                        if i == j:
                            output[index] = '1'
                    index = index +1
                #comparing interested up regulated gene set and file from .bson
                for x in my_new_list_dn:
                    for y in dict_data.get('dnGenes'):
                        if x == y:
                            output[index] = '-1'
                    index = index +1
                write.writerow(output)
